In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import requests as rq

In [ ]:
headers = {
'Accept': 'application/json',
}

uri = 'http://rest.genenames.org'
path = '/fetch/symbol/'
gene = 'SYNJ1'

response = rq.get(uri+path+gene, headers=headers)

data = response.json()
print 'Symbol: ' + data['response']['docs'][0]['symbol']
print 'ENSG: ' + data['response']['docs'][0]['ensembl_gene_id']

In [ ]:
working_dir = "/Users/filippo/Developer/tesi"
os.chdir(working_dir)
files = os.listdir("data_gtex")

In [ ]:
len(files)

In [ ]:
df=pd.read_csv("genes_hugo_symbol.txt", index_col=[0])
df.set_index('Hugo_Symbol', inplace=True)

In [ ]:
maxacceptables = 34
added = 1
for i,cfile in enumerate(files):
    if(re.match("[a-zA-Z0-9\-\.]{1,35}[.gz]",cfile)):
        print(cfile)
        cdf = pd.read_csv(("%s/data_gtex/%s"%(working_dir,cfile)), sep='\t', header=[0], index_col=[0], compression='gzip')
        try:
            df = df.combine_first(cdf.loc[df.index,cdf.keys()[2:]])
            added=added+1
        except:
            print "Unexpected error:", sys.exc_info()[0]
        print(df.columns)
    if added >= maxacceptables:
        break

print(added)

In [ ]:
df.fillna(value=0., inplace=True)

In [ ]:
df_conversion=pd.read_csv("hugo_ensg.txt", header=None)
df_conversion.head()

In [ ]:
#df.insert(0,'gene',df_conversion.loc[:,1].values)
#df.set_index('gene',inplace=True)
#df.drop('Hugo_Symbol', axis=1, inplace=True)

In [ ]:
# df = pd.read_csv(("%s/mainTable.csv"%working_dir))

In [ ]:
#df.columns=['gene', 'breast', 'kidney', 'bronchus', 'brain']

In [ ]:
df.head()

In [ ]:
print(("genes:%d\trealizations:%d"%(len(df.index),len(df.keys()))))

In [ ]:
df.to_csv(("%s/mainTable.csv"%working_dir))

In [ ]:
gene=df.index[0]
response = rq.get(uri+path+gene, headers=headers)

data = response.json()
print 'Symbol: ' + data['response']['docs'][0]['symbol']
print 'ENSG: ' + data['response']['docs'][0]['locus_group']
print 'ENSG: ' + data['response']['docs'][0]['ensembl_gene_id']

In [ ]:
df.loc[:,[k for k in df.keys() if 'TCGA' in k]].to_csv("mainTable_TCGA.csv")
df.loc[:,[k for k in df.keys() if 'GTEX' in k]].to_csv("mainTable_GTEX.csv")

## Zipf

In [ ]:
A = [df.loc[:,df.keys()[9]].values, df.loc[:,df.keys()[45]].values, df.loc[:,df.keys()[42]].values]
colors = ['red', 'blue', 'yellow']
fig = plt.figure()
x = np.arange(1, len(A[0]))
plt.ylabel("frequencyi")
plt.xlabel("RANKi")
plt.yscale('log')
plt.xscale('log')
plt.plot(x, 1./x, 'g--', label='$r^{-\\alpha}$')
for i,a in enumerate(A):
    plt.plot(np.sort(a/np.sum(a))[::-1], c=colors[i], label = 'file') 
plt.legend(fontsize=16)
plt.show()